In [83]:
import os
import pandas as pd
rootdir = '\\\\192.168.3.222\\user\\mov\\'
ltrimlen = len(rootdir)
pathcolname = 'path'
statecolname = 'state'
namecolname = 'name'
excelpath = 'D:\\myprojs\\github\\projs\\movie\\movie.xlsx'
htmltemplate = 'D:\\myprojs\\github\\projs\\movie\\movie.html.template'
htmlfile = 'D:\\myprojs\\github\\projs\\movie\\movie.html'
excelcols = [namecolname,'tag',pathcolname,statecolname]

def getmovfiles(files):
    ret = []
    for file in files:
        if file[-4:] in ('.ass', '.srt', '.txt', '.zip', '.jpg', '.sub', '.idx', '.rar', ):
            pass
        else:
            ret.append(file)
    return ret

def insert(df, item):
    return df.append({pathcolname:item}, ignore_index=True)

def scan():
    df = pd.DataFrame(columns=[pathcolname])
    for parent, dirnames, filenames in os.walk(rootdir):
        if '9.bad' in parent or '9.saw-jpg' in parent:
            continue
        if '#' == parent[-1]:  # 以#结尾的目录
            continue

        if len(dirnames) > 0 and len(filenames) > 0:
            for dir in dirnames:
                if '^' == dir[-1]:
                    continue
                df = insert(df, os.path.join(parent, dir)[ltrimlen:])
            movfiles = getmovfiles(filenames)
            for file in movfiles:
                df = insert(df, os.path.join(parent, file)[ltrimlen:])

        elif len(dirnames) > 0:
            for dir in dirnames:
                if '^' == dir[-1]:
                    continue
                df = insert(df, os.path.join(parent, dir)[ltrimlen:])

        elif len(filenames) > 0:
            if len(filenames) == 1:  # 只有一个文件
                continue
            movfiles = getmovfiles(filenames)  # 只有一个视频文件
            if len(movfiles) == 1:
                continue

            for file in movfiles:
                df = insert(df, os.path.join(parent, file)[ltrimlen:])
            
    return df.sort_values(by=[pathcolname])

def load():
    return pd.read_excel(excelpath)

def cmp(df_excel, df_dir):
    statecolname_dir = statecolname + '_dir'
    df_dir[statecolname_dir] = 'new'
    df_ret = df_excel.merge(df_dir, left_on=pathcolname, right_on=pathcolname, how='outer')

#     df_excel[statecolname] = 'new'
    df_ret[statecolname] = list(map(lambda state_dir, state_excel:
                        'new' if 'nan' == str(state_excel) else state_excel, 
                        df_ret[statecolname_dir], df_ret[statecolname]))
    df_ret[statecolname] = list(map(lambda state_dir, state_excel:
                        'del' if 'nan' == str(state_dir) else state_excel,
                        df_ret[statecolname_dir], df_ret[statecolname]))
    
    return df_ret

def save(df):
    df[excelcols].to_excel(excelpath,index=False)
    print(df[(df['state'] == 'new') | (df['state'] == 'del')])

def to_html(df):
    with open(htmltemplate, 'r') as f:
        html_temp = f.read()
        
    df_html = df[df[statecolname] != 'hide']
    df_html = df_html.fillna('')
    
    tags = []
    for t in set(df_html.tag.dropna()):
        tags += str(t).split(',')
    tags = list(set(tags))
    tags.sort()
    
    data_table = '\n'.join(df_html[[namecolname, 'tag', pathcolname]].to_html().split('\n')[1:])
    data_table += '<hr>' + ','.join(tags) + '<hr>\n'
    
    with open(htmlfile, 'w', encoding='utf-8') as f:
        f.write(html_temp.replace('%s', data_table))

In [84]:
def sync():
    df_dir = scan()
    df_excel = load()
    df_cmp = cmp(df_excel, df_dir)
    save(df_cmp)
    to_html(df_cmp)
sync()

Empty DataFrame
Columns: [name, tag, path, state, state_dir]
Index: []


In [75]:
to_html(load())

In [49]:
movfiles = []

def contains(item, strs):
    for str in strs:
        if str in item:
            return True
    return False
def ends(item, strs):
    for str in strs:
        if str == item[-len(str):]:
            return True
    return False

for fileitem in fileitems:
    if fileitem[-4:] in ('.ass', '.srt', '.txt', '.zip', '.jpg', '.sub', '.idx', '.rar', ):
        continue
    if contains(fileitem, ['中国好声音\\S', 's速度与激情\\s速度与激情',
            '聪明的一休.Disc', '.[丁丁历险记-', '\\f粉红小猪妹\\第', 'j机器猫\\doraemon', 'm猫和老鼠\\maohelaoshu', 'q七龙珠\\', 
            's神奇校车52集动画\\The Magic School Bus', 'y鼹鼠的故事\\ysdgshd','f粉红小猪妹\\','工作细胞.2018\\EP',
            '[爱探险的朵拉][Dora.The.Explorer]-',
            'c丛林大反攻\\丛林大反攻','g功夫熊猫\\功夫熊猫','g怪物史莱克\\怪物史瑞克','m马达加斯加\\马达加斯加',
            's狮子王\\狮子王','w玩具总动员\\玩具总动员','x小叮当\\小叮当','x驯龙高手\\驯龙高手',
            'x星球大战\\x星球大战',
            '奥运开幕式\\',
            '[宇宙：时空之旅].Cosmos.A.Spacetime.Odyssey.S01E', 'Prehistoric.America_ep', 'Planet.Dinosaur_ep', 'Digging.for.Britain.ep',
            'Spy.In.The.Wild.荒野间谍.纪录片-2017-[bbc]\\EP', '[BBC：数学的故事].BBC.The.Story.of.Maths.',
            '[中国通史].General.History.of.China.E','人体奥秘.ep','人间世 第','Frozen.Planet.EP','卢浮宫之旅.2009.Window.To.The.Louvre.EP',
            'Ancient.Worlds.ep','公視啊設計第','国家宝藏 ','The.Rise.of.Great.Nations.EP','The.Planets.ep','National.Treasure.In.Focus.EP',
            '我在故宫修文物.ep','故宫.The.Palace.Museum.ep','敦煌.ep','杰作背后的故事.ep','《河西走廊》第','[BBC：生命].BBC.Life.',
            '空中游欧陆.ep','舌尖上的中国_s01_e','蓝色星球-The.Blue.Planet-2001-[bbc]\\','行星地球-Planet.Earth-2006-[bbc]-CMCT\\',
            '诗词中国 ','[唯电影]《辉煌中国》 第','Yellowstone_ep','人类星球.human.planet-2011-[bbc]\\','园林-2014-[cctv]\\',
            '水果传-2018-[cctv]\\','Roof.of.the.World.EP',
            'd碟中谍CN\\碟中谍','d谍影重重\\谍影重重','d的士速递\\的士速递','d夺宝奇兵CN\\','c沉默的羔羊\\','b变形金刚CN\\变形金刚',
            'b博物馆奇妙夜CN\\博物馆奇妙夜','d达芬奇密码CN\\','z指环王CN\\指环王','z侏罗纪公园CN\\侏罗纪公园',
            'y异次元杀阵\\Cube','x笑傲江湖CN\\Swords man','w无间道CN\\无间道','q窃听风云CN\\窃听风云','q倩女幽魂CN\\倩女幽魂',
            'm木乃伊CN\\','j加勒比海盗CN\\','h黑衣人CN\\黑衣人','h黑客帝国\\The.Matrix','h霍比特人CN\\The.Hobbit',
            'h蝴蝶效应\\The.Butterfly.Effect','h虎胆龙威\\Die.Hard','h红白蓝\\Three.Colors.','h火天之城\\','h哈利波特CN\\',
            'g国家宝藏CN\\国家宝藏','f非诚勿扰CN\\','d大侦探福尔摩斯CN\\','d地心历险记CN\\','x修女也疯狂CN\\',
            'x喜爱夜蒲CN\\喜爱夜蒲','x猩球崛起CN\\','z终结者CN\\终结者',
            'j机动部队PTU\\机动部队','\\a暗战\\暗战','s神探夏洛克\\神探夏洛克',
            '纸牌屋-House.of.Cards\\S', '康熙王朝\\康熙王朝.E','乾隆王朝.E','24小时\\24小时','雍正王朝.E',
            '黑镜-Black.Mirror\\','基本演绎法-Elementary\\','新福尔摩斯-Sherlock\\',
            '高晓松mp3\\']):
        continue
    
    if ends(fileitem, ['1.cartoon','todo','1.movie','7.cancert','7.mv','8.cartoon-mv','Thumbs.db','9.bad',
            'mov\\cartoon','mov\\documentary','mov\\movie','mov\\tv',
            '【007】CN','【Marvel】CN','【周星驰】CN','【成龙】CN','【银河映像】CN']):
        continue
    
    movfiles.append(fileitem)
    
movfiles.sort()
movfiles

['\\\\192.168.3.222\\user\\mov\\1.cartoon\\Beyond.the.clouds.the.promised.place-云的彼端.约定的地方-2004-jp.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\Fantastic.Mr.Fox-了不起的狐狸爸爸-2009.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\Mary.and.Max-玛丽和马克思-2009-CMCT.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\Millennium.Actress-千年女优-2001-jp.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\The.Garden.of.Words-言叶之庭-2013-jp.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\The.light.of.the.fireflies.forest-萤火之森-2011-jp.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\les.triplettes.de.belleville-疯狂约会美丽都-2003-fr.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\todo\\The.Book.of.Life-生命之书-2014-[20fox]-CMCT#77.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\todo\\[亚瑟和他的迷你王国Ⅰ]国粤英三语.Arthur.and.the.Invisibles.2006.720p.BluRay.MKV.x264.AC3-CnSCG.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\todo\\公主与青蛙-The.Princess.And.The.Frog-2009-CHD#73.mkv',
 '\\\\192.168.3.222\\user\\mov\\1.cartoon\\todo\\埃及